In [1]:
## IMPORT NECESSARY PYTHON MODULES:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
## CONNECT TO SQL DATABASE:
# Provide database info
dbname = os.environ['dbname']
pswd = os.environ['pswd']
username = os.environ['username']
# construct a connection to a database
engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
# connect to the database
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [3]:
from mapbox import Geocoder

MAPBOX_ACCESS_TOKEN=os.environ['MAPBOX_ACCESS_TOKEN']

geocoder = Geocoder(access_token=MAPBOX_ACCESS_TOKEN)

#response_start = geocoder.forward("360 Elizabeth St, San Francisco, CA 94114")
response_start = geocoder.forward("501 Twin Peaks Blvd, San Francisco, CA 94114")
start_coords = response_start.geojson()['features'][0]['geometry']['coordinates']
lat1 = start_coords[0]
long1 = start_coords[1]

response_end = geocoder.forward("1720 Market St, San Francisco, CA 94102")
end_coords = response_end.geojson()['features'][0]['geometry']['coordinates']
lat2 = end_coords[0]
long2 = end_coords[1]

In [4]:
cur_get_ids_from_ll_start = con.cursor()
cur_get_ids_from_ll_end = con.cursor()
cur_get_ids_from_ll_start.execute("""select id from ways_vertices_pgr order by st_distance(the_geom, st_setsrid(st_makepoint(%s, %s), 4326)) limit 1;""", (lat1, long1))
cur_get_ids_from_ll_end.execute("""select id from ways_vertices_pgr order by st_distance(the_geom, st_setsrid(st_makepoint(%s, %s), 4326)) limit 1;""", (lat2, long2))

id_start = cur_get_ids_from_ll_start.fetchall()
id_end = cur_get_ids_from_ll_end.fetchall()

## Cell below needs editing to return cal_cost and length_ft columns. The totals from these columns then need to be calculated and displayed in the web app

In [5]:
curX = con.cursor()
curX.execute("""SELECT id, lon, lat, cost
   FROM ways_vertices_pgr
   JOIN
   (SELECT * FROM pgr_dijkstra('
    SELECT gid AS id,
         source,
         target,
         cal_cost AS cost,
         length_ft
        FROM ways_cals',
    %s, %s, directed := false )) AS route
   ON
   ways_vertices_pgr.id = route.node
   ORDER BY seq;""", (id_start, id_end))
Path = curX.fetchall()

## The following cell, given the cell above (which has been edited to return the cost column in the Path variable), gives the total of the "cal_cost" variable along the path. NOTE: this does NOT equal the calories burned!

In [6]:
Total_Cal = 0
for i in range(len(Path)):
    Total_Cal += Path[i][3]
    
Total_Cal    

12190.0

In [ ]:
Route_Coords = []
for i in range(len(Path)):
    Route_Coords.append([float(Path[i][1]),float(Path[i][2])])

len(Route_Coords)

In [ ]:
for i in range(len(Route_Coords)):
    Route_Coords[i][0], Route_Coords[i][1] = Route_Coords[i][1], Route_Coords[i][0]
    
#Route_Coords

In [ ]:
Route_str = "["
for i in range(len(Route_Coords)):
    Route_str += "["
    Route_str += str(Route_Coords[i][0])
    Route_str += ","
    Route_str += str(Route_Coords[i][1])
    Route_str += "]"
    Route_str += ","
Route_str = Route_str[:-1]
Route_str += "];"
    

In [ ]:
Route_str